### Базовое решение кейса "Рекомендательная система видео"
### Кейсодержатель: RUTUBE
#### Описание решения:

Приведенное базовое решение основано на подсчете совстречаемостей видео в сессии пользователя, или если говорить "по-бытовому": посчитаем как часто один товар попадает вместе с другим в одну корзину. На рисунке ниже вы можете видеть, что к пиву очень разумно рекомендовать рыбку, ведь ее частенько берут в дополенение к этому напитку.

Однако вполне реальной выглядит ситуация, что пиво встречается чаще в корзине с другим продуктом - с молоком. Почему так происходит? Молоко много с чем часто встречается в корзине просто потому что это очень популярный товар в магазине, в отличие от вяленой рыбки. Но было бы странно получать в рекомендациях к пиву молоко. Поэтому для расчета схожести применяется нормировка на популярность товаров! Далее на основе полученных для каждой пары товаров чисел, рекомендации можно отранжировать и получить в выдаче красоту, а не молоко:)

Подробнее читайте тут: https://www.inf.unibz.it/~ricci/ISR/papers/p293-davidson.pdf

![Рекомендации к пиву](fish_to_beer.jpg)

In [27]:
# from google.colab import drive
# drive.mount('/content/drive')

In [28]:
import pandas as pd
import math
import numpy as np
import zipfile
import os
import gc
import itertools
import scipy
from collections import defaultdict
# from google.colab import drive
from tqdm import tqdm

# drive.mount('/content/drive')

# print("loading data ...")
# hack = pd.read_csv('/content/drive/MyDrive/Цифровой прорыв/small_player_starts_train.csv')

In [29]:
hack = pd.read_csv('final_user_video_reaction_meta.csv')

C:\Users\bulae\AppData\Local\Temp\ipykernel_11540\2366008496.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  hack = pd.read_csv('final_user_video_reaction_meta.csv')


In [30]:
hack.head(10)

,Unnamed: 0,user_id,item_id,date_watch,watch_time,is_autorized,date_reaction,pos_emo,is_v_top,neg_emo,...,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
0,0,user_10000000,video_1487101,2023-08-21 20:25:47+03:00,2310,0,NaN,NaN,NaN,NaN,...,Премьера! В третьем сезоне экстремально-приклю...,Телепередачи,2022-12-05 11:00:20+03:00,5377.480,75496,0,0.294598,0.260142,0.237338,0.252909
1,1,user_10000000,video_222807,2023-08-21 19:06:33+03:00,4611,0,NaN,NaN,NaN,NaN,...,«Четыре свадьбы» отправляются в Турцию! Невест...,Телепередачи,2023-08-10 21:19:06+03:00,6919.000,16723,0,0.000000,0.000000,0.025105,0.039709
2,2,user_10000033,video_1761620,2023-08-21 07:02:56+03:00,196,0,NaN,NaN,NaN,NaN,...,В прямом эфире Соловьёв LIVE – новости и размы...,Телепередачи,2022-10-14 12:06:53+03:00,70972.000,343030,0,0.000000,0.000000,0.000000,0.000000
3,3,user_10000087,video_1106848,2023-08-21 03:55:46+03:00,490,0,NaN,NaN,NaN,NaN,...,"ЙОУ БЕЗРАБОТНЫЕ, это полное прохождение НОВОГО...",Видеоигры,2023-08-17 13:57:07+03:00,1059.917,19764,0,0.000000,0.000000,0.000000,0.010326
4,4,user_10000087,video_633280,2023-08-21 04:04:26+03:00,1,0,NaN,NaN,NaN,NaN,...,Atomic Heart - «Инстинкт Истребления» Прохожд...,Видеоигры,2023-08-14 10:00:20+03:00,3286.529,59,0,0.000000,0.000000,0.000000,0.011585
5,5,user_10000092,video_2137593,2023-08-21 17:40:31+03:00,52,0,NaN,NaN,NaN,NaN,...,"Всем привет! Приветствую вас на своем канале ""...",Эзотерика,2023-08-12 09:16:18+03:00,283.240,28896,0,0.000000,0.000000,0.000000,0.000478
6,6,user_10000099,video_2224179,2023-08-21 09:25:44+03:00,171,1,NaN,NaN,NaN,NaN,...,NaN,Кулинария,2023-08-16 19:02:11+03:00,179.240,1,0,0.000000,0.000000,0.000000,0.000000
7,7,user_10000099,video_30396,2023-08-21 09:29:18+03:00,102,1,2023-08-21 06:29:26+03:00,Like,0.0,NaN,...,Как приготовить рваную говядину на углях. Реце...,Кулинария,2023-08-17 13:10:00+03:00,922.854,1410,0,0.000000,0.000000,0.000000,0.000000
8,8,user_10000099,video_856096,2023-08-21 09:22:38+03:00,172,1,NaN,NaN,NaN,NaN,...,NaN,Кулинария,2023-08-21 09:21:58+03:00,176.360,1,0,0.000000,0.000000,0.000000,0.000000
9,9,user_10000115,video_283933,2023-08-21 12:19:19+03:00,2493,0,NaN,NaN,NaN,NaN,...,Премьера! «Выжить в Дубае» – новое масштабное ...,Телепередачи,2023-08-20 21:00:23+03:00,5616.280,75496,0,0.000000,0.000000,0.000000,0.572971


Группы видео

Предобработка данных, вычислений значения по составленной формуле для каждой записи просмотра пользователя

In [31]:
POS_EMO_COEF = 1.2
V_TOP_COEF = 1.3
MAX_VALUE = 2.7

hack['is_v_top'] = np.where(hack['is_v_top'] == 1, V_TOP_COEF, 1)
hack['pos_emo'] = np.where(hack['pos_emo'].isna(), 1, POS_EMO_COEF)
hack['val'] = ((hack['watch_time']/hack['duration']) * hack['is_v_top'] * hack['pos_emo'] + hack['ctr.CTR_10days_21_08'])/MAX_VALUE
hack_val = hack[['user_id','item_id','val','category_title']].fillna(0)
hack_val.head()

,user_id,item_id,val,category_title
0,user_10000000,video_1487101,0.252770,Телепередачи
1,user_10000000,video_222807,0.261531,Телепередачи
2,user_10000033,video_1761620,0.001023,Телепередачи
3,user_10000087,video_1106848,0.175047,Видеоигры
4,user_10000087,video_633280,0.004404,Видеоигры


Выполняем обработку полученных значений, убирая выбросы, изменяя значения всех записей больше 1 на 0.9. А также заполняем поля с названием шоу для элементов, не являющимися шоу, названиями видео

In [32]:
hack[['tv_title']] = hack[['tv_title']].fillna(0)
hack['tv_title'] = np.where(hack['tv_title'] == 0, hack['video_title'], hack['tv_title'])
hack['val'] = np.where(hack['val'] > 1, 0.9, hack['val'])
hack

,Unnamed: 0,user_id,item_id,date_watch,watch_time,is_autorized,date_reaction,pos_emo,is_v_top,neg_emo,...,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08,val
0,0,user_10000000,video_1487101,2023-08-21 20:25:47+03:00,2310,0,NaN,1.0,1.0,NaN,...,Телепередачи,2022-12-05 11:00:20+03:00,5377.480,75496,0,0.294598,0.260142,0.237338,0.252909,0.252770
1,1,user_10000000,video_222807,2023-08-21 19:06:33+03:00,4611,0,NaN,1.0,1.0,NaN,...,Телепередачи,2023-08-10 21:19:06+03:00,6919.000,16723,0,0.000000,0.000000,0.025105,0.039709,0.261531
2,2,user_10000033,video_1761620,2023-08-21 07:02:56+03:00,196,0,NaN,1.0,1.0,NaN,...,Телепередачи,2022-10-14 12:06:53+03:00,70972.000,343030,0,0.000000,0.000000,0.000000,0.000000,0.001023
3,3,user_10000087,video_1106848,2023-08-21 03:55:46+03:00,490,0,NaN,1.0,1.0,NaN,...,Видеоигры,2023-08-17 13:57:07+03:00,1059.917,19764,0,0.000000,0.000000,0.000000,0.010326,0.175047
4,4,user_10000087,video_633280,2023-08-21 04:04:26+03:00,1,0,NaN,1.0,1.0,NaN,...,Видеоигры,2023-08-14 10:00:20+03:00,3286.529,59,0,0.000000,0.000000,0.000000,0.011585,0.004404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006759,2006759,user_9999954,video_1541155,2023-08-21 10:06:17+03:00,22,0,NaN,1.0,1.0,NaN,...,Здоровье,2023-08-20 12:01:48+03:00,104.042,3,0,0.000000,0.000000,0.000000,0.000000,0.078316
2006760,2006760,user_9999962,video_2177785,2023-08-21 11:03:36+03:00,21,0,NaN,1.0,1.0,NaN,...,Образование,2022-10-27 11:08:40+03:00,255.616,33330,0,0.063337,0.038339,0.053981,0.086685,0.062533
2006761,2006761,user_9999974,video_380634,2023-08-21 10:56:28+03:00,33,1,NaN,1.0,1.0,NaN,...,Техника и оборудование,2023-08-14 21:32:59+03:00,40.209,0,0,0.000000,0.000000,0.000000,0.000000,0.303967
2006762,2006762,user_9999977,video_291331,2023-08-21 09:29:39+03:00,1,0,NaN,1.0,1.0,NaN,...,Разное,2022-03-03 17:16:35+03:00,30.322,1421,0,0.000000,0.000000,0.500000,0.000000,0.012215


Формируем списки с названиями категорий для каждого типа категорий: Шоу, Каналы, Категория, Видео

In [33]:
titles_group_by_tv = hack['tv_title'].unique()
titles_group_by_author = hack['author_title'].unique()
titles_group_by_category = hack['category_title'].unique()
titles_group_by_video = hack['item_id'].unique()


Далее формируем 4 словаря для каждого из типов категорий с топами для каждой категории

In [34]:
dict_top_by_tv = {}
hack_tv = hack.groupby(['tv_title', 'item_id']).agg({'val': 'mean'})['val'].sort_values(ascending=False)

for tv in tqdm(titles_group_by_tv):
    dict_top_by_tv[tv] = hack_tv[tv]

dict_top_by_tv[titles_group_by_tv[0]]

100%|██████████| 217396/217396 [01:32<00:00, 2356.78it/s]


item_id
video_1151688    0.363691
video_1800914    0.358682
video_1013107    0.325619
video_1102367    0.325165
video_853304     0.324742
video_1557144    0.324655
video_170733     0.320835
video_1090723    0.314101
video_1317715    0.309986
video_2111866    0.307480
video_2189663    0.306908
video_1903845    0.304848
video_1063784    0.304345
video_1174960    0.303626
video_550582     0.302222
video_659912     0.301165
video_881440     0.298794
video_585801     0.297764
video_1589958    0.297735
video_2288234    0.296274
video_1371446    0.295155
video_1735904    0.294369
video_1308201    0.290251
video_1515026    0.289547
video_923879     0.288680
video_2014114    0.288554
video_1560572    0.288421
video_1291980    0.286359
video_297693     0.284675
video_1331804    0.284374
video_851792     0.284316
video_1501477    0.282428
video_1487101    0.282336
video_2116008    0.280079
video_2311904    0.277826
video_64837      0.277474
video_769286     0.277332
video_1095484    0.275187
vide

In [26]:
dict_top_by_author = {}
hack_author = hack.groupby(['author_title', 'item_id']).agg({'val': 'mean'})['val'].sort_values(ascending=False)

for a in tqdm(titles_group_by_author):
    dict_top_by_author[a] = hack_author[a]

dict_top_by_author[titles_group_by_author[0]]

100%|██████████| 45040/45040 [00:21<00:00, 2122.77it/s]


item_id
video_61656      0.900000
video_656237     0.731724
video_1393481    0.707408
video_122730     0.697987
video_1658211    0.654237
                   ...   
video_1329374   -0.000864
video_1428896   -0.002062
video_1837673   -0.002503
video_1537654   -0.003035
video_168127    -0.012986
Name: val, Length: 1415, dtype: float64

In [36]:
dict_top_by_category = {}
hack_category = hack.groupby(['category_title', 'item_id']).agg({'val': 'mean'})['val'].sort_values(ascending=False)

for c in tqdm(titles_group_by_category):
    dict_top_by_category[c] = hack_category[c]

dict_top_by_category[titles_group_by_category[0]]

100%|██████████| 45/45 [00:00<00:00, 562.49it/s]


item_id
video_2244365    0.991093
video_1238978    0.989862
video_2005171    0.989219
video_1280753    0.988554
video_2069325    0.985699
                   ...   
video_2063407   -0.018475
video_2097861   -0.021387
video_1741085   -0.024518
video_1420524         NaN
video_88718           NaN
Name: val, Length: 25670, dtype: float64

In [38]:
dict_top_by_video = hack.groupby('item_id').agg({'val': 'mean'})['val'].sort_values(ascending=False)

Ниже функция рекомендации

Объявляем константы

In [39]:
NUM_REC_FOR_AUTHOR = 4
NUM_REC_FOR_TV = 6
NUM_REC_FOR_CATEGORY = 0
const_for_rec_type = {"author_title": NUM_REC_FOR_AUTHOR, "tv_title": NUM_REC_FOR_TV, "category_title": NUM_REC_FOR_CATEGORY}
dict_top_for_rec_type = {"author_title": dict_top_by_author, "tv_title": dict_top_by_tv, "category_title": dict_top_by_category, "item_id": dict_top_by_video}

Создаем функцию, которая будет подсчитывать для категории количество видео, которые ожидаются в рекомендациях

In [40]:
def create_users_rec_type(user, count_video, rec_type):
    users = {}
    C = const_for_rec_type[rec_type]

    users[user] = hack[hack['user_id'] == user][['item_id', rec_type]].groupby(rec_type).count().sort_values('item_id', ascending=False).to_dict()
    users[user]['item_id'] = {k: round(v/count_video * C) for k, v in users[user]['item_id'].items()}

    return users

Создаем функцию, которая будет рекомендовать видео в соответствии с сформированным словарем количества видео на категорию

In [41]:
def recomend_video(user, count_video, rec_type):
    rec_video = []
    rec_val = []
    users = create_users_rec_type(user, count_video, rec_type)

    for k, v in users[user]['item_id'].items():
        if v == 0:
            break
        temp = dict_top_for_rec_type[rec_type][k]
        drop_el = pd.merge(hack[hack['user_id'] == user][['item_id']], temp, how='inner', on=['item_id'])['item_id'].values
        temp = temp.drop(columns='item_id', index=drop_el)
        rec_video.extend(list(temp[0:v].index))
        rec_val.extend(list(temp[0:v]))

    return (rec_video, rec_val)

Создаем функцию, которая будет добавлять в набор рекомендательных видео, элементы до необходимого значения

In [42]:
def recomend_missing_video(user, count_rec_author, count_rec_tv, count_video, rec_type):
    rec_category_video = []
    rec_category_val = []
    v = NUM_REC_FOR_AUTHOR + NUM_REC_FOR_TV - count_rec_author - count_rec_tv
    users = create_users_rec_type(user, count_video, rec_type)

    for k in users[user]['item_id'].keys():
        if v > 0:
            temp = dict_top_for_rec_type[rec_type][k]
            drop_el = pd.merge(hack[hack['user_id'] == user][['item_id']], temp, how='inner', on=['item_id'])['item_id'].values
            temp = temp.drop(columns='item_id', index=drop_el)
            rec_category_video.extend(list(temp[0:v].index))
            rec_category_val.extend(list(temp[0:v]))
            v = NUM_REC_FOR_AUTHOR + NUM_REC_FOR_TV - count_rec_author - count_rec_tv - len(rec_category_video)
        else:
            break

    return (rec_category_video, rec_category_val)

Создаем функцию, которая формирует список рекомендаци по нескольким типам категорий в соответствии с заранее заданным числом видео на тип категорий и объединяет их в один список, полученный набор может содержать повторяющиеся видео

In [43]:
def ununique_recomend_video(user):
    count_video = hack[hack['user_id'] == user]['item_id'].size
    rec_author_video, rec_author_val = recomend_video(user, count_video, 'author_title')
    rec_tv_video, rec_tv_val = recomend_video(user, count_video, 'tv_title')
    rec_category_video, rec_category_val = recomend_missing_video(user, len(rec_author_video), len(rec_tv_video), count_video, 'category_title')
    rec_all_video = rec_author_video
    rec_all_video.extend(rec_category_video)
    rec_all_video.extend(rec_tv_video)

    rec_all_val = rec_author_val
    rec_all_val.extend(rec_category_val)
    rec_all_val.extend(rec_tv_val)

    return (rec_all_video, rec_all_val)

Создаем функцию, которая будет составлять список только с уникальными рекомендациям выше описанной функции

In [44]:
def unique_recomend_video(user):
    rec_all_video, rec_all_val = ununique_recomend_video(user)
    rec = [k for k, v in sorted(zip(rec_all_video, rec_all_val), key=lambda item: item[1], reverse=True)]

    return list(set(rec[:10]))

In [45]:
user_watch = hack[hack['user_id'] == 'user_9996000']['video_title'].unique()
print("Пользователь user_9996000 посмотрел 21.07:")
for w in user_watch:
    print('\t' + w)

Пользователь user_9996000 посмотрел 21.07:
	Stand Up: Андрей Колмачевский - высокая тревожность
	Енот в шоке)))
	Каха, Серго
	Comedy Club: Муж олень | Демис Карибидис, Марина Кравец
	STAND UP, 1 сезон, 2 выпуск (эфир 29.09.2013)
	de156fd0a1bf4121bea46a9ff4f7f1ed.mp4
	Выжить в Дубае, 9 выпуск
	Хорошего дня
	STAND UP, 1 сезон, 1 выпуск (эфир 22.09.2013)
	Котэ. Все мимо))


In [46]:
recomend = unique_recomend_video('user_9996000')
title_recomend = []
for v in recomend:
    title_recomend.extend(hack[hack['item_id'] == v]['video_title'].unique())

print("Рекомендуем пользователю user_9996000:")
for t in range(len(title_recomend)-1, -1, -1):
    print('\t' + title_recomend[t])

Рекомендуем пользователю user_9996000:
	Камеди Клаб «Отгоните машину» Гарик Харламов
	туалет
	Stand Up: Иван Усович - О секс-приложении, имени Нина и людях, которые играют в онлайн-казино
	Comedy Баттл. Суперсезон - Большов (финал) 26.12.2014
	Борис Краснов в Comedy Club (28.04.2017)
	А вы посчитали?
	Выжить в Дубае
	Бабульки научат вас как пить пиво
	Stand Up: Слава Комиссаренко - О пьющих девушках, своих родителях и приколах в подъезде


In [49]:
submission = pd.read_csv("sample_submission.csv")
user_for_pred = submission['user_id'].values
top = list(dict_top_by_video[:10].index)

for i, u in tqdm(enumerate(user_for_pred[67000:69000])):
    r = list(unique_recomend_video(u))
    if r == []:
        submission['recs'][i] = str([top])
    else:
        submission['recs'][i] = str([r])

submission.head(10)

2000it [16:08,  2.07it/s]


,user_id,recs
0,user_26511551,"[['video_303263', 'video_1067394', 'video_2143..."
1,user_29194819,"[['video_1763730', 'video_61656', 'video_65623..."
2,user_29734049,"[['video_303263', 'video_1067394', 'video_2143..."
3,user_955460,"[['video_303263', 'video_1067394', 'video_2143..."
4,user_7065521,"[['video_1363115', 'video_2214393', 'video_231..."
5,user_27113563,"[['video_303263', 'video_1067394', 'video_2143..."
6,user_10757579,"[['video_303263', 'video_1067394', 'video_2143..."
7,user_27968546,"[['video_303263', 'video_1067394', 'video_2143..."
8,user_9113975,"[['video_303263', 'video_1067394', 'video_2143..."
9,user_23303152,"[['video_303263', 'video_1067394', 'video_2143..."


Выгрузка рекомендаций

In [50]:
submission.to_csv("submission67k_to_69k.csv", index=False)
# !cp submission10k_D.csv "drive/My Drive/"

In [ ]:
res = pd.merge(submission_0, submission_1, how='left',on='user_id')
res['recs'] = np.where(len(res['recs_y']) > 2, res['recs_y'], res['recs_x'])
res = res.drop(columns=['recs_x', 'recs_y'])